# P2I-7 - MODELISATION NUMERIQUE POUR L’INGENIEUR 

# SUJET N°5 - Capteur MEMS résonant pour la mesure de masse de ultra-haute précision 
 
Contact : sebastien.baguet@insa-lyon.fr 

## Contexte 

Les importantes avancées dans la micro-fabrication des M/NEMS (Micro et Nano Electro-Mechanical Systems) 
de ces dernières années ont apporté des développements conséquents dans la navigation, les télécommunications, 
les jeux vidéo, etc.  
Après les MEMS, l’émergence des NEMS (Nano Electro Mechanical Systems) a ouvert la porte à des gammes 
entières de nouvelles applications. En effet ces nanosystèmes ayant des dimensions critiques de l’ordre de quelques 
dizaines de nanomètres, et donc une masse et une raideur extrêmement réduites, montrent des sensibilités à des 
perturbations extérieures (ajout de masse, modification de propriétés chimiques, effort extérieur...) inégalées. En 
grande majorité résonants, les NEMS permettent en particulier de quantifier l’ajout d’une masse en mesurant leur 
variation de fréquence propre induite.
 
PHOTO ?
Fig.1 – Nano-poutre encastrée-libre avec électrode d’actionnement et nano-fils piezorésistifs de détection 
(Cliché du CEA-LETI). 
 
Un résonateur est constitué d’une nano-poutre (voir Fig. 1), équipée classiquement d’au moins une électrode 
pour l'actionnement. Une tension continue Vdc fléchit la poutre alors qu'une tension alternative harmonique Vac la 
sollicite sur sa fréquence fondamentale de flexion. Classiquement il s'agit de faire vibrer le résonateur en flexion sur 
sa fréquence fondamentale, c-à-d. au maximum d'amplitude. Une perturbation extérieure change la raideur ou la 
masse du résonateur et donc sa fréquence fondamentale. Plus la taille du résonateur est petite, plus la fréquence 
fondamentale est grande (de l’ordre de quelques MHz au GHz pour des capteurs NEMS) et plus la sensibilité de ces 
dispositifs à une perturbation est grande, induisant de meilleures performances. 
Particule m  


In [ ]:
# import des librairies additionnelles
import numpy as np
from numpy import zeros
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp, odeint
import time
#%matplotlib notebook
#%matplotlib widget
#%matplotlib ipympl

hello
